# ![Snowflake connector](../images/database/snowflake_connector_logo.png)
This example walks through the basics of reading and writing data with the Ray Snowflake connector.


## Initialize ray
Ray will automatically be initialized with defaults when calling any ray or ray dataset methods. To specify configuration, add the below. 

In [1]:
import ray, logging
logging.basicConfig(level=logging.ERROR) # only show errors
logging.getLogger('snowflake.connector').setLevel(logging.ERROR)

if not ray.is_initialized():
    ray.init()

2023-02-27 21:04:15,854	INFO worker.py:1242 -- Using address localhost:9031 set in the environment variable RAY_ADDRESS
find: ‘.git’: No such file or directory
2023-02-27 21:04:16,132	INFO worker.py:1360 -- Connecting to existing Ray cluster at address: 10.0.36.75:9031...
2023-02-27 21:04:16,138	INFO worker.py:1548 -- Connected to Ray cluster. View the dashboard at https://console.anyscale.com/api/v2/sessions/ses_vnmb5jgl4z6q98h61dx25rccju/services?redirect_to=dashboard 
2023-02-27 21:04:16,141	INFO packaging.py:330 -- Pushing file package 'gcs://_ray_pkg_de75729c0244af0b2679b245ffd05193.zip' (0.26MiB) to Ray cluster...
2023-02-27 21:04:16,143	INFO packaging.py:343 -- Successfully pushed file package 'gcs://_ray_pkg_de75729c0244af0b2679b245ffd05193.zip'.


## Connection properties
The Snowflake connection properties need to be provided to the data source upon creation. The minimal required properties are `user`, `password`, `account` and `warehouse`. To use API keys instead of a password, functionality to load Snowflake API keys is also provided. API keys can be loaded from a file specified by the `private_key_file` property, or can be passed directly via the `private_key` property. If the key is password protected, the password can be given via the `pk_password` property.  Optional properties like database and schema can also be provided at construction or be included in the fully specified table name of format `db.schema.table` when calling read or write operations with a table or subquery.

Below is an example of loading properties from the environment, and filtering them by the 'SNOWFLAKE_' prefix.

In [2]:
import os

# get properties form env
env_connect_props = {
    key.replace('SNOWFLAKE_','').lower(): value 
    for key,value in os.environ.items() if 'SNOWFLAKE_' in key
}

# add db and schema in connect props
connect_props = dict(
    database = 'SNOWFLAKE_SAMPLE_DATA',
    schema = 'TPCH_SF1',
    warehouse='COMPUTE_WH',
    **env_connect_props
)

print('Connection properties:')
print('\n'.join(connect_props.keys()))

Connection properties:
database
schema
warehouse
account
private_key_file
pk_password
user


# Reading from Snowflake
Ray will use Snowflake optimizations that allow query results to be read in parallel into a Ray cluster. The created Ray datasets is composed of Pandas dataframes that are spread across the Ray cluster to allow for the distributed operations required in machine learning.

![Snowflake read table](../images/database/snowflake_read.png)

### Read from tables
In order to read an entire table into a a Ray cluster, utilize the Ray data `read_snowflake` method. The code below will read in a sample table from the Snowflake sample database.

In [3]:
from ray.data import read_snowflake

# read the entire table
ds = read_snowflake(connect_props, table='CUSTOMER') 

# display the first 3 results
print('count:',ds.count())
ds.limit(3).to_pandas()

count: 150000


Read progress: 100%|██████████| 1/1 [00:00<00:00,  2.81it/s]


,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
0,30001,Customer#000030001,"Ui1b,3Q71CiLTJn4MbVp,,YCZARIaNTelfst",4,14-526-204-4500,8848.47,MACHINERY,frays wake blithely enticingly ironic asymptote
1,30002,Customer#000030002,UVBoMtILkQu1J3v,11,21-340-653-9800,5221.81,MACHINERY,he slyly ironic pinto beans wake slyly above t...
2,30003,Customer#000030003,CuGi9fwKn8JdR,21,31-757-493-7525,3014.89,BUILDING,e furiously alongside of the requests. evenly ...


### Read with a query
For more control over columns and rows read, as well as joining data from multiple tables, a query can be specified instead of a table name. 

In [4]:
QUERY = 'SELECT C_ACCTBAL, C_MKTSEGMENT FROM CUSTOMER WHERE C_ACCTBAL < 1000'

# read the result of the query
ds2 = read_snowflake(connect_props, query=QUERY)

# display the first 3 results
print('count:',ds2.count())
ds2.limit(3).to_pandas()

count: 27119


Read progress: 100%|██████████| 1/1 [00:00<00:00,  2.39it/s]


,C_ACCTBAL,C_MKTSEGMENT
0,711.56,BUILDING
1,121.65,AUTOMOBILE
2,794.47,HOUSEHOLD


### Additional read parameters
For reading from Snowflake, underlying Python API arguments are also available. The `timeout` and `params` arguments may be used in the [cursor execute method](https://docs.snowflake.com/en/user-guide/python-connector-api.html#object-cursor).

The code below uses the params to specify params to be used by Snowflake when executing the query.

In [5]:
QUERY = 'SELECT C_ACCTBAL, C_MKTSEGMENT FROM CUSTOMER WHERE C_ACCTBAL > ?'

ds3 = read_snowflake(connect_props, query=QUERY, params=[1000], timeout=1000)
print('count:',ds3.count())
ds3.limit(3).to_pandas()

count: 122881


Read progress: 100%|██████████| 1/1 [00:00<00:00,  2.90it/s]


,C_ACCTBAL,C_MKTSEGMENT
0,8848.47,MACHINERY
1,5221.81,MACHINERY
2,3014.89,BUILDING


## Writing
The Ray Snowflake connector will use Snowflake API to write each partition of data in parallel. Each partition of data in the Ray dataset will have a write task that writes in parallel to Snowflake. 
![Snowflake write table](../images/database/snowflake_write.png)

### Write to tables
In order to write a dataset into Snowflake table, use the `write_snowflake` method of the dataset object. Repartition the dataset in order to set the number of write tasks.

First, a new database and table needs to be created using the Ray Snowflake Connector or the native Snowflake API. 

> Note: When using the Ray data connector, we can use the API key loading functionality built into the connector class.

In [6]:
from ray.data.datasource import SnowflakeConnector

write_connect_props = {
    **connect_props, 
    'database':'RAY_SAMPLE', 
    'schema':'PUBLIC'
}
with SnowflakeConnector(**write_connect_props) as con:
    # create destination database
    con.query(f'CREATE DATABASE IF NOT EXISTS RAY_SAMPLE')
    con.commit()
    
    # create destination table
    con.query('''
        CREATE OR REPLACE TABLE CUSTOMER_COPY 
        LIKE SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER
    ''')

The example below writes the previously read data into a new database table that are created using the Snowflake Python API.

In [7]:
# write the dataset to the table 
ds.write_snowflake(
    write_connect_props, 
    table='CUSTOMER_COPY'
)

# read the new table
ds4 = read_snowflake(write_connect_props, table='CUSTOMER_COPY')
print('count:',ds4.count())
ds4.limit(3).to_pandas()

2023-02-27 21:04:28,323	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[read->write]
read->write: 100%|██████████| 19/19 [00:13<00:00,  1.43it/s]


count: 150000


Read progress: 100%|██████████| 1/1 [00:00<00:00,  3.73it/s]
2023-02-27 21:04:54,212	WARNING plan.py:528 -- Warning: The Ray cluster currently does not have any available CPUs. The Dataset job will hang unless more CPUs are freed up. A common reason is that cluster resources are used by Actors or Tune trials; see the following link for more details: https://docs.ray.io/en/master/data/dataset-internals.html#datasets-and-tune


,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
0,87868,Customer#000087868,uPZ8cB41EapBpopIsIFmDo,1,11-552-716-1697,1049.32,FURNITURE,platelets affix quickly after the boldly iron...
1,87869,Customer#000087869,nmpIurFWrzzqZgXGlIRdlDkXpJkEd,17,27-765-230-3208,825.60,AUTOMOBILE,ke regular accounts. express ac
2,87870,Customer#000087870,kzLemw38LYB0djEi9kYMbUgQsgHiCCJG,8,18-436-876-3101,2827.08,BUILDING,regular asymptotes according to the slyly fin...


### Additional write parameters
For writing to Snowflake, the native Snowflake API arguments are also available from the [write_pandas](https://docs.snowflake.com/en/user-guide/python-connector-api.html#module-snowflake-connector-pandas-tools) method. The following is a list of the parameters that may be useful:

- `auto_create_table`: When true, will automatically create a table with corresponding columns for each column in the passed in DataFrame. The table will not be created if it already exists
- `overwrite`: When true, and if auto_create_table is true, then it drops the table. Otherwise, it truncates the table. In both cases it will replace the existing contents of the table with that of the passed in Pandas DataFrame.
- `table_type`: The table type of to-be-created table. The supported table types include ``temp``/``temporary`` and ``transient``. Empty means permanent table as per SQL convention.

In the example below, we use the `auto_create_table` parameter to create the output table before writing.

In [8]:
# write the dataset to the table, using an autocreated table
ds.write_snowflake(
    write_connect_props, 
    table='CUSTOMER_COPY_2',
    auto_create_table=True
)

# read the new table
ds5 = read_snowflake(write_connect_props, table='CUSTOMER_COPY_2')
print('count:',ds5.count())
ds5.limit(3).to_pandas()

2023-02-27 21:04:54,322	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[read->write]
read->write: 100%|██████████| 19/19 [00:21<00:00,  1.13s/it]


count: 3150000


Read progress: 100%|██████████| 1/1 [00:08<00:00,  8.25s/it]


,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
0,6167,Customer#000006167,oPGRaBt5IuSxtQhwgztJyv1miL,7,17-798-361-3738,6729.32,FURNITURE,ly final requests grow alongside of t
1,6168,Customer#000006168,"TfeEEZ9Ds8x3eQljQUYaNzpu6tE1ap,JGRn2gZm",24,34-877-467-2443,6633.98,MACHINERY,ily. special platelets about the slyly
2,6169,Customer#000006169,UDeLABnhksFK7tfInLsDPRtynoWxesfiJ,7,17-656-209-2093,-681.05,HOUSEHOLD,y final pinto beans. slyly even dependencies b...


## Advanced Usage
If more low level access to the Ray Snowflake connector is needed, the underlying `SnowflakConnector` and `SnowflakeDatasource` can be used.

### Snowflake Connector
The `SnowflakeConnector` class holds the connection properties and logic required to establish a connection with Snowflake. Internally it calls the native Python Snowflake API in order to read and write from and to Snowflake tables in parallel across the cluster. The datasource uses the Snowflake Python API's optimized `read_batch` and `write_pandas` methods to enable parallel read and writes of data.

The connector is also a Python context manager, and utilize `with` semantics to define when a connection should be established, db operations commited to the database, and the connection closed. 

The code below will read from a sample table using the connector to manage the connection.

In [9]:
from ray.data.datasource import SnowflakeConnector

# query the number of rows, using the connection context to
# manage transactions
with SnowflakeConnector(**connect_props) as con:
    count = con.query_int(f'SELECT COUNT(*) FROM CUSTOMER')

print(count)

150000


Alternatively, you can use `try` blocks with the connector's `open`, `commit` and `close` methods. 

In [10]:
connector = SnowflakeConnector(**connect_props)
try:
    connector.open()
    count = connector.query_int(f'SELECT COUNT(*) FROM CUSTOMER')
finally:
    connector.close()
    
print(count)

150000


### Snowflake Datasource
The Snowflake datasource can be used with the Ray data `read_datasource` and `write_datasource` methods to read and write to Snowflake databases using the distibuted processing capabilities of Ray data. The datasource uses a SnowflakeConnector class that is derived from the DBAPI2Connector class. 

Below is an exmaple of creating the datasource using the previously defined connect properties, and then using it to read and write.

In [11]:
from ray.data import read_datasource
from ray.data.datasource import SnowflakeDatasource

# use read_datasource to read
ds = read_datasource(
    SnowflakeDatasource(connector), 
    table='CUSTOMER'
)
 
ds.limit(3).to_pandas()

Read progress: 100%|██████████| 1/1 [00:00<00:00,  3.31it/s]


,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
0,30001,Customer#000030001,"Ui1b,3Q71CiLTJn4MbVp,,YCZARIaNTelfst",4,14-526-204-4500,8848.47,MACHINERY,frays wake blithely enticingly ironic asymptote
1,30002,Customer#000030002,UVBoMtILkQu1J3v,11,21-340-653-9800,5221.81,MACHINERY,he slyly ironic pinto beans wake slyly above t...
2,30003,Customer#000030003,CuGi9fwKn8JdR,21,31-757-493-7525,3014.89,BUILDING,e furiously alongside of the requests. evenly ...


In [12]:
# use write_datasource to write
connector = SnowflakeConnector(**write_connect_props)
datasource = SnowflakeDatasource(connector)
ds.write_datasource(
    datasource, 
    table='CUSTOMER_3',
    auto_create_table=True
)

2023-02-27 21:05:31,473	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[read->write]
read->write: 100%|██████████| 19/19 [00:13<00:00,  1.37it/s]


### DML and DDL
The connector can also be used for any DDL or DML operations you would normally execute through the Snowflake Python API. These operations just pass through to the underlying Snowflake API. 

The code below will create the objects needed for writing to tables. Note that a commit is issued between the queries so the DDL operation executes prior to the next one that is dependent. An alternative is to use two `with` blocks to define transaction boundaries.

In [13]:
with connector as con:
    con.query(f'CREATE DATABASE IF NOT EXISTS RAY')
    con.commit()
    con.query(f'''
        CREATE OR REPLACE TABLE RAY.PUBLIC.CUSTOMER_COPY
            LIKE SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER
    ''')

### Pandas data mapping
The Snowflake Datasource converts Pandas data types using the Snowflake Python Connector API. Data mappings are available from the Snowflake [documentation](https://docs.snowflake.com/en/user-guide/python-connector-pandas.html#snowflake-to-pandas-data-mapping). 

The below code is an example of reading and writing all the available data formats.

In [14]:
with connector as con:
    con.query("""
        CREATE OR REPLACE TABLE SAMPLE_TABLE (
            ID INT,
            SAMPLE_NUMBER NUMBER(6,2),
            SAMPLE_DECIMAL DECIMAL(8,3),
            SAMPLE_FLOAT FLOAT,
            SAMPLE_VARCHAR VARCHAR,
            SAMPLE_BINARY BINARY,
            SAMPLE_INT INT,
            SAMPLE_DATE DATE,
            SAMPLE_TIME TIME,
            SAMPLE_TIMESTAMP_TZ TIMESTAMP_TZ,
            SAMPLE_TIMESTAMP_NTZ TIMESTAMP_NTZ,
            SAMPLE_TIMESTAMP_LTZ TIMESTAMP_LTZ,
            SAMPLE_GEOGRAPHY GEOGRAPHY,
            SAMPLE_VARIANT VARIANT,
            SAMPLE_ARRAY ARRAY,
            SAMPLE_OBJECT OBJECT
        )
    """)
    con.commit()
    con.query("""
        INSERT INTO SAMPLE_TABLE 
        VALUES (
            0,
            1111.11,
            22222.222,
            3.333333333,
            '4444444444',
            '01ffeeddaa',
            6666,
            TO_DATE('2007-07-07'),
            TO_TIME('08:00:00.000'),
            TO_TIMESTAMP_TZ('2009-07-08 08:00:00'),
            TO_TIMESTAMP_NTZ('2010-07-08 08:00:00.000'),
            TO_TIMESTAMP_LTZ('2011-07-08 08:00:00.000'),
            'POINT(-122.35 37.55)',
            NULL,
            NULL,
            NULL
        )
    """)
    con.query("""UPDATE SAMPLE_TABLE SET SAMPLE_VARIANT = to_variant(parse_json('{"key3": "value3", "key4": "value4"}'))""")
    con.query("UPDATE SAMPLE_TABLE SET SAMPLE_ARRAY = [1,'two',3,4]")
    con.query("UPDATE SAMPLE_TABLE SET SAMPLE_OBJECT = {'thirteen':13, 'zero':0}")

sample = read_snowflake(write_connect_props, table='SAMPLE_TABLE')
sample.to_pandas()

2023-02-27 21:05:49,311	WARNING read_api.py:353 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.
Read progress: 100%|██████████| 1/1 [00:00<00:00,  3.93it/s]


,ID,SAMPLE_NUMBER,SAMPLE_DECIMAL,SAMPLE_FLOAT,SAMPLE_VARCHAR,SAMPLE_BINARY,SAMPLE_INT,SAMPLE_DATE,SAMPLE_TIME,SAMPLE_TIMESTAMP_TZ,SAMPLE_TIMESTAMP_NTZ,SAMPLE_TIMESTAMP_LTZ,SAMPLE_GEOGRAPHY,SAMPLE_VARIANT,SAMPLE_ARRAY,SAMPLE_OBJECT
0,0,1111.11,22222.222,3.333333,4444444444,b'\x01\xff\xee\xdd\xaa',6666,2007-07-07,08:00:00,2009-07-08 08:00:00-07:00,2010-07-08 08:00:00,2011-07-08 08:00:00-07:00,"{\n ""coordinates"": [\n -122.35,\n 37.55...","{\n ""key3"": ""value3"",\n ""key4"": ""value4""\n}","[\n 1,\n ""two"",\n 3,\n 4\n]","{\n ""thirteen"": 13,\n ""zero"": 0\n}"


The below code writes the sample data back to Snowflake:

In [15]:
new_sample = sample.drop_columns(['SAMPLE_BINARY']) # binary column write does not work in Snowflake API
new_sample.write_snowflake(
    write_connect_props, 
    table='SAMPLE_TABLE_DEST', 
    auto_create_table=True
)
read_snowflake(
    write_connect_props, 
    table='SAMPLE_TABLE_DEST'
).to_pandas()

2023-02-27 21:05:49,635	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[read->MapBatches(<lambda>)->write]
read->MapBatches(<lambda>)->write: 100%|██████████| 1/1 [00:02<00:00,  2.41s/it]
2023-02-27 21:05:53,799	WARNING read_api.py:353 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.
Read progress: 100%|██████████| 1/1 [00:00<00:00,  4.02it/s]


,ID,SAMPLE_NUMBER,SAMPLE_DECIMAL,SAMPLE_FLOAT,SAMPLE_VARCHAR,SAMPLE_INT,SAMPLE_DATE,SAMPLE_TIME,SAMPLE_TIMESTAMP_TZ,SAMPLE_TIMESTAMP_NTZ,SAMPLE_TIMESTAMP_LTZ,SAMPLE_GEOGRAPHY,SAMPLE_VARIANT,SAMPLE_ARRAY,SAMPLE_OBJECT
0,0,1111.11,22222.222,3.333333,4444444444,6666,2007-07-07,08:00:00,2009-07-08 15:00:00,1278576000000000,2011-07-08 15:00:00,"{\n ""coordinates"": [\n -122.35,\n 37.55...","{\n ""key3"": ""value3"",\n ""key4"": ""value4""\n}","[\n 1,\n ""two"",\n 3,\n 4\n]","{\n ""thirteen"": 13,\n ""zero"": 0\n}"
1,0,1111.11,22222.222,3.333333,4444444444,6666,2007-07-07,08:00:00,2009-07-08 15:00:00,1278576000000000,2011-07-08 15:00:00,"{\n ""coordinates"": [\n -122.35,\n 37.55...","{\n ""key3"": ""value3"",\n ""key4"": ""value4""\n}","[\n 1,\n ""two"",\n 3,\n 4\n]","{\n ""thirteen"": 13,\n ""zero"": 0\n}"
2,0,1111.11,22222.222,3.333333,4444444444,6666,2007-07-07,08:00:00,2009-07-08 15:00:00,1278576000000000,2011-07-08 15:00:00,"{\n ""coordinates"": [\n -122.35,\n 37.55...","{\n ""key3"": ""value3"",\n ""key4"": ""value4""\n}","[\n 1,\n ""two"",\n 3,\n 4\n]","{\n ""thirteen"": 13,\n ""zero"": 0\n}"
3,0,1111.11,22222.222,3.333333,4444444444,6666,2007-07-07,08:00:00,2009-07-08 15:00:00,1278576000000000,2011-07-08 15:00:00,"{\n ""coordinates"": [\n -122.35,\n 37.55...","{\n ""key3"": ""value3"",\n ""key4"": ""value4""\n}","[\n 1,\n ""two"",\n 3,\n 4\n]","{\n ""thirteen"": 13,\n ""zero"": 0\n}"
4,0,1111.11,22222.222,3.333333,4444444444,6666,2007-07-07,08:00:00,2009-07-08 15:00:00,1278576000000000,2011-07-08 15:00:00,"{\n ""coordinates"": [\n -122.35,\n 37.55...","{\n ""key3"": ""value3"",\n ""key4"": ""value4""\n}","[\n 1,\n ""two"",\n 3,\n 4\n]","{\n ""thirteen"": 13,\n ""zero"": 0\n}"
5,0,1111.11,22222.222,3.333333,4444444444,6666,2007-07-07,08:00:00,2009-07-08 15:00:00,1278576000000000,2011-07-08 15:00:00,"{\n ""coordinates"": [\n -122.35,\n 37.55...","{\n ""key3"": ""value3"",\n ""key4"": ""value4""\n}","[\n 1,\n ""two"",\n 3,\n 4\n]","{\n ""thirteen"": 13,\n ""zero"": 0\n}"
6,0,1111.11,22222.222,3.333333,4444444444,6666,2007-07-07,08:00:00,2009-07-08 15:00:00,1278576000000000,2011-07-08 15:00:00,"{\n ""coordinates"": [\n -122.35,\n 37.55...","{\n ""key3"": ""value3"",\n ""key4"": ""value4""\n}","[\n 1,\n ""two"",\n 3,\n 4\n]","{\n ""thirteen"": 13,\n ""zero"": 0\n}"
7,0,1111.11,22222.222,3.333333,4444444444,6666,2007-07-07,08:00:00,2009-07-08 15:00:00,1278576000000000,2011-07-08 15:00:00,"{\n ""coordinates"": [\n -122.35,\n 37.55...","{\n ""key3"": ""value3"",\n ""key4"": ""value4""\n}","[\n 1,\n ""two"",\n 3,\n 4\n]","{\n ""thirteen"": 13,\n ""zero"": 0\n}"
8,0,1111.11,22222.222,3.333333,4444444444,6666,2007-07-07,08:00:00,2009-07-08 15:00:00,1278576000000000,2011-07-08 15:00:00,"{\n ""coordinates"": [\n -122.35,\n 37.55...","{\n ""key3"": ""value3"",\n ""key4"": ""value4""\n}","[\n 1,\n ""two"",\n 3,\n 4\n]","{\n ""thirteen"": 13,\n ""zero"": 0\n}"
9,0,1111.11,22222.222,3.333333,4444444444,6666,2007-07-07,08:00:00,2009-07-08 15:00:00,1278576000000000,2011-07-08 15:00:00,"{\n ""coordinates"": [\n -122.35,\n 37.55...","{\n ""key3"": ""value3"",\n ""key4"": ""value4""\n}","[\n 1,\n ""two"",\n 3,\n 4\n]","{\n ""thirteen"": 13,\n ""zero"": 0\n}"
